# Metadata

**Buildings metadata**

* building_id: building code-name with the structure SiteID_SimplifiedUsage_UniqueName.
* site_id: animal-code-name for the site.
* building_id_kaggle: building ID used for the Kaggle competition (numeric).
* site_id_kaggle: site ID used for the Kaggle competition (numeric).
* primaryspaceusage: Primary space usage of all buildings is mapped using the energystar scheme building description types.
* sub_primaryspaceusage: energystar scheme building description types subcategory.
* sqm: Floor area of building in square meters (m2).
* lat: Latitude of building location to city level.
* lng: Longitude of building location to city level.
* timezone: site's timezone.
* electricity: presence of this kind of meter in the building. Yes if affirmative, NaN if negative.
* hotwater: presence of this kind of meter in the building. Yes if affirmative, NaN if negative.
* chilledwater: presence of this kind of meter in the building. Yes if affirmative, NaN if negative.
* steam: presence of this kind of meter in the building. Yes if affirmative, NaN if negative.
* water: presence of this kind of meter in the building. Yes if affirmative, NaN if negative.
* irrigation: presence of this kind of meter in the building. Yes if affirmative, NaN if negative.
* solar: presence of this kind of meter in the building. Yes if affirmative, NaN if negative.
* gas: presence of this kind of meter in the building. Yes if affirmative, NaN if negative.
* industry: Industry type corresponding to building.
* subindustry: More detailed breakdown of Industry type corresponding to building.
* heatingtype: Type of heating in corresponding building.
* yearbuilt: Year corresponding to when building was first constructed, in the format YYYY.
* date_opened: Date building was opened for use, in the format D/M/YYYY.
* numberoffloors: Number of floors corresponding to building.
* occupants: Usual number of occupants in the building.
* energystarscore: Rating of building corresponding to building energystar scheme (Energy Star Score).
* eui: Energy use intensity of the building (kWh/year/m2).
* site_eui: Energy (Consumed/Purchased) use intensity of the site (kWh/year/m2).
* source_eui: Total primary energy consumption normalized by area (Takes into account conversion efficiency of primary energy into secondary energy).
* leed_level: LEED rating of the building (Leadership in Energy and Environmental Design), most widely used green building rating system.
* rating: Other building energy ratings.


In [1]:
# Importing the required libraries 
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
plt.rcParams.update({'font.family': 'Arial'})

import matplotlib as mpl
import matplotlib.pylab as pylab
%matplotlib inline
import seaborn as sns
sns.set_style("darkgrid")
mpl.style.use('ggplot')

import geopandas as gpd
from shapely.geometry import Point, Polygon
import gc

In [2]:
# Specifying the filepath for the data 
path = "../data/metadata/"

In [3]:
# Buildings MetaData data
df_metadata = pd.read_csv(path + "metadata.csv")

# Inspecting the some columns of the data 
df_metadata.sample(5)

,building_id,site_id,building_id_kaggle,site_id_kaggle,primaryspaceusage,sub_primaryspaceusage,sqm,sqft,lat,lng,...,yearbuilt,date_opened,numberoffloors,occupants,energystarscore,eui,site_eui,source_eui,leed_level,rating
618,Rat_health_Shane,Rat,556.0,3.0,Healthcare,Hospital (General Medical & Surgical),41806.4,450000.0,38.903504,-77.005349,...,2012.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
659,Bear_education_Val,Bear,594.0,4.0,Education,Education,16469.9,177281.0,37.871903,-122.260729,...,1964.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1598,Cockatoo_lodging_Linwood,Cockatoo,1418.0,15.0,Lodging/residential,Dormitory,3837.1,41302.0,42.459837,-76.485292,...,1975.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
251,Fox_education_Leona,Fox,217.0,2.0,Education,College Classroom,26286.5,282946.0,33.424425,-111.928140,...,2003.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169,Robin_public_Carolina,Robin,138.0,1.0,Public services,Library,10984.0,118231.0,51.518790,-0.134556,...,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,C


In [4]:
# Inspecting the columns of the metadata
df_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1636 entries, 0 to 1635
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   building_id            1636 non-null   object 
 1   site_id                1636 non-null   object 
 2   building_id_kaggle     1449 non-null   float64
 3   site_id_kaggle         1599 non-null   float64
 4   primaryspaceusage      1615 non-null   object 
 5   sub_primaryspaceusage  1615 non-null   object 
 6   sqm                    1636 non-null   float64
 7   sqft                   1636 non-null   float64
 8   lat                    1399 non-null   float64
 9   lng                    1399 non-null   float64
 10  timezone               1636 non-null   object 
 11  electricity            1578 non-null   object 
 12  hotwater               185 non-null    object 
 13  chilledwater           555 non-null    object 
 14  steam                  370 non-null    object 
 15  wate

## EDA

In [5]:
# shape of the metadata
df_metadata.shape

(1636, 32)

The dataset has information on the metadata of 1636 buildings.

In [6]:
# Delete the Kaggle IDs since this data was reproduced for a Kaggle Competion 
df_metadata = df_metadata.drop(columns=['building_id_kaggle', 'site_id_kaggle'])

### Missing Values

In [7]:
# Percentage of missing values in each feature
missing_features = [features for features in df_metadata.columns if df_metadata[features].isnull().sum() > 0]
for features in missing_features:
    # Updated to round the missing values percentage to two decimal places
    print(f"{features} : {round(df_metadata[features].isnull().mean() * 100, 2)}%")

primaryspaceusage : 1.28%
sub_primaryspaceusage : 1.28%
lat : 14.49%
lng : 14.49%
electricity : 3.55%
hotwater : 88.69%
chilledwater : 66.08%
steam : 77.38%
water : 91.08%
irrigation : 97.74%
solar : 99.69%
gas : 89.18%
industry : 64.61%
subindustry : 64.61%
heatingtype : 86.86%
yearbuilt : 50.06%
date_opened : 98.72%
numberoffloors : 73.04%
occupants : 85.94%
energystarscore : 90.04%
eui : 81.72%
site_eui : 90.04%
source_eui : 90.04%
leed_level : 99.02%
rating : 88.75%


In [8]:
# Features with less than 10% missing values will be retained for analysis
missing_pct = df_metadata.isnull().mean().mul(100)
features_lt10 = missing_pct[missing_pct < 10].sort_values()

print("Features with <10% missing values:")
print(list(features_lt10.index))

# Also show their missing percentages
features_lt10.round(2).to_frame(name='missing_%')

Features with <10% missing values:
['building_id', 'site_id', 'sqm', 'sqft', 'timezone', 'primaryspaceusage', 'sub_primaryspaceusage', 'electricity']


,missing_%
building_id,0.00
site_id,0.00
sqm,0.00
sqft,0.00
timezone,0.00
primaryspaceusage,1.28
sub_primaryspaceusage,1.28
electricity,3.55


In addition to building_id, site_id, sqm and primaryspaceusage, lat, lon and yearbuilt will be retained for the analysis. Eventhough yearbuilt, lat and lon have more than ten percent missing values, they might contain valuable information. 

In [9]:
# keep_features = ['building_id', 'site_id', 'sqm', 'primaryspaceusage']
keep_features = ['building_id', 'site_id', 'sqm', 'primaryspaceusage', 'lat', 'lng', 'yearbuilt']
df_metadata = df_metadata[keep_features]
df_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1636 entries, 0 to 1635
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   building_id        1636 non-null   object 
 1   site_id            1636 non-null   object 
 2   sqm                1636 non-null   float64
 3   primaryspaceusage  1615 non-null   object 
 4   lat                1399 non-null   float64
 5   lng                1399 non-null   float64
 6   yearbuilt          817 non-null    float64
dtypes: float64(4), object(3)
memory usage: 89.6+ KB


In [10]:
# convert yearbuilt to integer data type 
df_metadata['yearbuilt'] = df_metadata[df_metadata['yearbuilt'].notna()]['yearbuilt'].astype('Int64')
df_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1636 entries, 0 to 1635
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   building_id        1636 non-null   object 
 1   site_id            1636 non-null   object 
 2   sqm                1636 non-null   float64
 3   primaryspaceusage  1615 non-null   object 
 4   lat                1399 non-null   float64
 5   lng                1399 non-null   float64
 6   yearbuilt          817 non-null    Int64  
dtypes: Int64(1), float64(3), object(3)
memory usage: 91.2+ KB


In [11]:
df_metadata.head()  

,building_id,site_id,sqm,primaryspaceusage,lat,lng,yearbuilt
0,Panther_lodging_Dean,Panther,508.8,Lodging/residential,28.517689,-81.379039,1989
1,Panther_lodging_Shelia,Panther,929.0,Lodging/residential,28.517689,-81.379039,1992
2,Panther_lodging_Ricky,Panther,483.1,Lodging/residential,28.517689,-81.379039,2016
3,Panther_education_Rosalie,Panther,690.5,Education,28.517689,-81.379039,2008
4,Panther_education_Misty,Panther,252.7,Education,28.517689,-81.379039,2004


In [12]:
# # Infer country from lat/lng and add a 'country' column to df_metadata
# url_countries = "https://raw.githubusercontent.com/nvkelso/natural-earth-vector/master/geojson/ne_110m_admin_0_countries.geojson"
# world = gpd.read_file(url_countries)[['ADMIN', 'geometry']].rename(columns={'ADMIN': 'country'}).to_crs('EPSG:4326')

# mask = df_metadata['lat'].notna() & df_metadata['lng'].notna()
# df_metadata['country'] = np.nan  # initialize

# points = gpd.GeoDataFrame(
#     df_metadata.loc[mask, ['building_id', 'lat', 'lng']],
#     geometry=gpd.points_from_xy(df_metadata.loc[mask, 'lng'], df_metadata.loc[mask, 'lat']),
#     crs='EPSG:4326'
# )

# try:
#     joined = gpd.sjoin(points, world, how='left', predicate='within')[['building_id', 'country']]
#     country_map = joined.set_index('building_id')['country']
#     df_metadata.loc[mask, 'country'] = df_metadata.loc[mask, 'building_id'].map(country_map)
# except Exception:
#     # Fallback without spatial index
#     def point_to_country(row):
#         pt = Point(row['lng'], row['lat'])
#         hit = world[world.contains(pt)]
#         return hit['country'].iloc[0] if not hit.empty else np.nan

#     df_metadata.loc[mask, 'country'] = df_metadata.loc[mask].apply(point_to_country, axis=1)

# # quick check
# df_metadata.head().head()

In [12]:
import geopandas as gpd
from shapely.geometry import Point
import numpy as np

# Load world map once outside the function/script if possible
url_countries = "https://raw.githubusercontent.com/nvkelso/natural-earth-vector/master/geojson/ne_110m_admin_0_countries.geojson"
world = gpd.read_file(url_countries)[['ADMIN', 'geometry']].rename(columns={'ADMIN': 'country'}).to_crs('EPSG:4326')

def get_country_from_lat_lng(df_metadata):
    """Infers country from lat/lng and adds a 'country' column."""
    mask = df_metadata['lat'].notna() & df_metadata['lng'].notna()
    
    # Create GeoDataFrame for points only for rows with valid coordinates
    points_to_process = df_metadata.loc[mask].copy()
    points_to_process['geometry'] = gpd.points_from_xy(points_to_process['lng'], points_to_process['lat'])
    points_gdf = gpd.GeoDataFrame(points_to_process, crs='EPSG:4326')

    # Perform spatial join efficiently
    joined = gpd.sjoin(points_gdf, world, how='left', predicate='within')
    
    # Map results back to the original DataFrame
    df_metadata['country'] = np.nan
    df_metadata.loc[mask, 'country'] = joined['country'].values
    
    return df_metadata

# Example usage (assuming df_metadata is defined)
df_metadata = get_country_from_lat_lng(df_metadata)
df_metadata.head()

/var/folders/ds/cj7pjk114y31h97nf7898zqc0000gn/T/ipykernel_5718/3286257323.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['United States of America' 'United States of America'
 'United States of America' ... 'United Kingdom' 'United Kingdom'
 'United Kingdom']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_metadata.loc[mask, 'country'] = joined['country'].values


,building_id,site_id,sqm,primaryspaceusage,lat,lng,yearbuilt,country
0,Panther_lodging_Dean,Panther,508.8,Lodging/residential,28.517689,-81.379039,1989,United States of America
1,Panther_lodging_Shelia,Panther,929.0,Lodging/residential,28.517689,-81.379039,1992,United States of America
2,Panther_lodging_Ricky,Panther,483.1,Lodging/residential,28.517689,-81.379039,2016,United States of America
3,Panther_education_Rosalie,Panther,690.5,Education,28.517689,-81.379039,2008,United States of America
4,Panther_education_Misty,Panther,252.7,Education,28.517689,-81.379039,2004,United States of America


In [18]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time


def get_region_from_coords(coordinates):
    """
    Uses reverse geocoding to get city and state from latitude and longitude.
    """
    try:
        # Define a user agent as required by Nominatim's terms of service
        geolocator = Nominatim(user_agent="bdg2_project_app", timeout=10)
        
        # Reverse geocode the coordinates
        location = geolocator.reverse(coordinates, addressdetails=True, language='en')
        
        if location and 'address' in location.raw:
            address = location.raw['address']
            city = address.get('city', address.get('town', address.get('village', 'Unknown City')))
            state = address.get('state', 'Unknown State')
            county = address.get('county', 'Unknown County')
            return f"{city}, {county}, {state}"
        else:
            return "Location not found"
    except Exception as e:
        return f"Error: {e}"

# Only process rows with valid coordinates
mask_valid = df_metadata['lat'].notna() & df_metadata['lng'].notna()

# Use RateLimiter to respect the API rate limits (1 request per second is safe)
geolocator = Nominatim(user_agent="bdg2_project_app", timeout=10)
reverse_geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1)

# Initialize region with a default value
df_metadata['region'] = "Unknown Region"

# Apply reverse geocoding using (lat, lng) tuples to valid rows only
def safe_reverse(lat, lng):
    try:
        return reverse_geocode((lat, lng))
    except Exception:
        return None

locations = df_metadata.loc[mask_valid, ['lat', 'lng']].apply(lambda r: safe_reverse(r['lat'], r['lng']), axis=1)

# Map results back to the dataframe
df_metadata.loc[mask_valid, 'region'] = locations.apply(
    lambda loc: (
        f"{loc.raw['address'].get('city', loc.raw['address'].get('town', loc.raw['address'].get('village', '')))}, "
        f"{loc.raw['address'].get('state', '')}"
    ) if loc else "Unknown Region"
)

# Print the results
print(df_metadata[['site_id', 'lat', 'lng', 'region']])

      site_id        lat        lng                   region
0     Panther  28.517689 -81.379039         Orlando, Florida
1     Panther  28.517689 -81.379039         Orlando, Florida
2     Panther  28.517689 -81.379039         Orlando, Florida
3     Panther  28.517689 -81.379039         Orlando, Florida
4     Panther  28.517689 -81.379039         Orlando, Florida
...       ...        ...        ...                      ...
1631    Mouse  51.521939  -0.120069  Greater London, England
1632    Mouse  51.521939  -0.120069  Greater London, England
1633    Mouse  51.521939  -0.120069  Greater London, England
1634    Mouse  51.521939  -0.120069  Greater London, England
1635    Mouse  51.521939  -0.120069  Greater London, England

[1636 rows x 4 columns]


In [20]:
df_metadata.head()

,building_id,site_id,sqm,primaryspaceusage,lat,lng,yearbuilt,country,nuts2_id,nuts2_name,site_nuts2_id,coordinates,region
0,Panther_lodging_Dean,Panther,508.8,Lodging/residential,28.517689,-81.379039,1989,United States of America,NaN,NaN,NaN,"28.5176885788321, -81.3790387999998","Orlando, Florida"
1,Panther_lodging_Shelia,Panther,929.0,Lodging/residential,28.517689,-81.379039,1992,United States of America,NaN,NaN,NaN,"28.5176885788321, -81.3790387999998","Orlando, Florida"
2,Panther_lodging_Ricky,Panther,483.1,Lodging/residential,28.517689,-81.379039,2016,United States of America,NaN,NaN,NaN,"28.5176885788321, -81.3790387999998","Orlando, Florida"
3,Panther_education_Rosalie,Panther,690.5,Education,28.517689,-81.379039,2008,United States of America,NaN,NaN,NaN,"28.5176885788321, -81.3790387999998","Orlando, Florida"
4,Panther_education_Misty,Panther,252.7,Education,28.517689,-81.379039,2004,United States of America,NaN,NaN,NaN,"28.5176885788321, -81.3790387999998","Orlando, Florida"


In [19]:
df_metadata['country'].value_counts()

country
United States of America    1128
United Kingdom               215
Netherlands                   36
Canada                        20
Name: count, dtype: int64

In [47]:
# keep_features = ['building_id', 'site_id', 'sqm', 'primaryspaceusage', 'region', 'yearbuilt', 'country']
df_metadata = df_metadata[['building_id', 'site_id', 'sqm', 'primaryspaceusage', 'yearbuilt', 'region', 'country']]
df_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1636 entries, 0 to 1635
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   building_id        1636 non-null   object 
 1   site_id            1636 non-null   object 
 2   sqm                1636 non-null   float64
 3   primaryspaceusage  1615 non-null   object 
 4   yearbuilt          817 non-null    Int64  
 5   region             1399 non-null   object 
 6   country            1399 non-null   object 
dtypes: Int64(1), float64(1), object(5)
memory usage: 91.2+ KB


In [ ]:
df_metadata.loc[df_metadata['country'].isna(), 'region'] = np.nan

In [50]:
df_metadata.loc[df_metadata['country'].isna(), 'region'] = np.nan

In [51]:
df_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1636 entries, 0 to 1635
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   building_id        1636 non-null   object 
 1   site_id            1636 non-null   object 
 2   sqm                1636 non-null   float64
 3   primaryspaceusage  1615 non-null   object 
 4   yearbuilt          817 non-null    Int64  
 5   region             1399 non-null   object 
 6   country            1399 non-null   object 
dtypes: Int64(1), float64(1), object(5)
memory usage: 91.2+ KB


In [58]:
mask = df_metadata['region'].isna() & df_metadata['country'].isna()
df_metadata.loc[mask, ['building_id','site_id','region','country']]

,building_id,site_id,region,country
931,Gator_office_Carrie,Gator,NaN,NaN
932,Gator_office_Lucy,Gator,NaN,NaN
933,Gator_assembly_Lelia,Gator,NaN,NaN
934,Gator_other_Elfriede,Gator,NaN,NaN
935,Gator_assembly_Maurine,Gator,NaN,NaN
...,...,...,...,...
1500,Eagle_assembly_Portia,Eagle,NaN,NaN
1501,Eagle_assembly_Ian,Eagle,NaN,NaN
1502,Eagle_assembly_Lacy,Eagle,NaN,NaN
1503,Eagle_office_Sonya,Eagle,NaN,NaN


In [60]:
# Check for missing (True) and non-missing (False) values in 'region' grouped by 'site_id'
df_metadata.groupby('site_id')['country'].apply(lambda x: x.isna().value_counts()).unstack().fillna(0)

,False,True
site_id,,
Bear,92.0,0.0
Bobcat,0.0,36.0
Bull,124.0,0.0
Cockatoo,124.0,0.0
Crow,5.0,0.0
Eagle,0.0,106.0
Fox,137.0,0.0
Gator,0.0,74.0
Hog,163.0,0.0


In [52]:
# Generate the cleaned file for metadata analysis
df_metadata.to_csv(path + "metadata_analysis.csv", index=False)

In [53]:
# Load the cleaned file to verify
df = pd.read_csv(path + "metadata_analysis.csv")
df.head()

,building_id,site_id,sqm,primaryspaceusage,yearbuilt,region,country
0,Panther_lodging_Dean,Panther,508.8,Lodging/residential,1989.0,"Orlando, Florida",United States of America
1,Panther_lodging_Shelia,Panther,929.0,Lodging/residential,1992.0,"Orlando, Florida",United States of America
2,Panther_lodging_Ricky,Panther,483.1,Lodging/residential,2016.0,"Orlando, Florida",United States of America
3,Panther_education_Rosalie,Panther,690.5,Education,2008.0,"Orlando, Florida",United States of America
4,Panther_education_Misty,Panther,252.7,Education,2004.0,"Orlando, Florida",United States of America


In [54]:
df[df['site_id'] == 'Bull'].head()

,building_id,site_id,sqm,primaryspaceusage,yearbuilt,region,country
1005,Bull_office_Lilla,Bull,16890.2,Office,NaN,"Austin, Texas",United States of America
1006,Bull_lodging_Caren,Bull,16509.6,Lodging/residential,NaN,"Austin, Texas",United States of America
1007,Bull_education_Clarice,Bull,5132.0,Education,NaN,"Austin, Texas",United States of America
1008,Bull_office_Hilton,Bull,743.6,Office,NaN,"Austin, Texas",United States of America
1009,Bull_lodging_Leonard,Bull,3831.4,Lodging/residential,NaN,"Austin, Texas",United States of America


In [55]:
df['site_id'].value_counts()

site_id
Rat         305
Hog         163
Lamb        147
Fox         137
Panther     136
Cockatoo    124
Bull        124
Eagle       106
Bear         92
Gator        74
Robin        52
Peacock      47
Bobcat       36
Wolf         36
Swan         21
Moose        15
Shrew         9
Mouse         7
Crow          5
Name: count, dtype: int64

In [30]:
df['site_id'].nunique()

19